Directories

In [ ]:
import os
import glob
import shutil

In [ ]:
source_dir = '/Users/aaron/test/'
target_dir = '/Users/aaron/target/'
extensions = ['.tif','.shp', '.shx', '.dbf']

image_dir = target_dir + 'image/'
mask_dir = target_dir + 'mask/'

In [ ]:
# Create folders for images and masks
try:
    os.mkdir(image_dir)
    print("Directory " , image_dir ,  " Created ")
except FileExistsError:
    print("Directory " , image_dir ,  " already exists")
try:
    os.mkdir(mask_dir)
    print("Directory " , mask_dir ,  " Created ")
except FileExistsError:
    print("Directory " , mask_dir ,  " already exists")

Copy images and masks to folders

In [ ]:
#counter=1

for foldername, subfolders, filenames in os.walk(source_dir):
    print('Searching files in %s...' % (foldername))
    #counter+=1
    for filename in filenames:
        if filename.endswith('.tif'):
            totalCopyPath = os.path.join(foldername, filename)
            newName = os.path.join(image_dir, str(counter)+'.tif')
            print(totalCopyPath)
            print('Copying ' + filename + ' to ' + newName)
            shutil.copy(totalCopyPath, image_dir)
            #os.rename(image_dir+filename, newName)
        elif filename.endswith(tuple(extensions)):
            totalCopyPath = os.path.join(foldername, filename)
            newName = os.path.join(mask_dir, str(counter))
            print(newName)
            print(totalCopyPath)
            print('Copying ' + filename + ' to ' + newName)
            shutil.copy(totalCopyPath, mask_dir)
            #os.rename(mask_dir+filename, '1')
print('Done.')

In [ ]:
# Go through subfolders of source directory and copy every .tif file into 'target_dir/images/' folder
counter=1
for foldername, subfolders, filenames in os.walk(source_dir):
    print('Searching files in %s...' % (foldername))
    for filename in filenames:
        if filename.endswith('.tif'):
            totalCopyPath = os.path.join(foldername, filename)
            newName = str(counter) +'.tif'
            counter+=1
            print('Copying ' + filename + ' to ' + newName)
            shutil.copy(totalCopyPath, image_dir)
print('Done.')

Create smaller images

In [ ]:
import cv2
from matplotlib import pyplot as plt
from patchify import patchify, unpatchify

In [ ]:
# Read image
large_image = cv2.imread('../data/original_data/image/image.tif', 0)
patches = patchify(large_image, (512, 512), step=512)
print("Large image shape is: ", large_image.shape)
print("Patches array shape is: ", patches.shape)

In [ ]:
# Create patches
for i in range(patches.shape[0]):
    for j in range(patches.shape[1]):
        single_patch_img = patches[i, j]
        if not cv2.imwrite('../data/tiled_data/patched_images/' + 'image_' + '_'+ str(i)+'_'+str(j)+'.tif', single_patch_img):
            raise Exception("Could not write the image")

In [ ]:
# Read mask
large_image = cv2.imread('../data/original_data/masks/mask.tif', 0)
patches = patchify(large_image, (512, 512), step=512)
print("Large image shape is: ", large_image.shape)
print("Patches array shape is: ", patches.shape)

In [ ]:
# Create patches
for i in range(patches.shape[0]):
    for j in range(patches.shape[1]):
        single_patch_img = patches[i, j]
        if not cv2.imwrite('../data/tiled_data/patched_masks/' + 'image_' + '_' + str(i) + '_' + str(j) + '.tif', single_patch_img):
            raise Exception("Could not write the image")

Cleanup